<a href="https://colab.research.google.com/github/Catatau52/Data_Mining-Data_from_CVM/blob/main/Personal_Project_Data_Mining_(CVM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carregando as bibliotecas

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=9e2f13ef139cdf0a43345313947e82d224a8c264291b0f9d264957b1384a0d51
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import pandas as pd
import numpy as np
import wget
import decimal
import os
from zipfile import ZipFile

#Acessando a base de dados da CVM

download da base itr (trimestral) e dfp (anual) desde 2011 até 2022

In [ ]:
url_base_itr = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
url_base_dfp = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

Criando uma lista com todos os arquivos e baixando

In [ ]:
arquivos_itr_zip = []
arquivos_dfp_zip = []
for ano in range(2011,2022+1):
  arquivos_itr_zip.append(f'itr_cia_aberta_{ano}.zip')
  arquivos_dfp_zip.append(f'dfp_cia_aberta_{ano}.zip')


In [ ]:
for arq in arquivos_itr_zip:
  wget.download(url_base_itr+arq)

for arq in arquivos_dfp_zip:
  wget.download(url_base_dfp+arq)

In [ ]:
for arq in arquivos_itr_zip:
  ZipFile(arq, 'r').extractall('ITR')

for arq in arquivos_dfp_zip:
  ZipFile(arq, 'r').extractall('DFP')

Concatenando todos os resultados em ITR e DFP e gerando CSV

In [ ]:
# Criar a pasta
os.makedirs(os.path.join('/content', 'Dados'), exist_ok=True)

docs = ['BPA_ind','BPP_ind','DRE_ind','DVA_ind']

for doc in docs:
  arquivo = pd.DataFrame()
  for ano in range(2011,2022+1):
    arquivo = pd.concat([arquivo, pd.read_csv(f'ITR/itr_cia_aberta_{doc}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'Dados/itr_cia_aberta_{doc}_2011-2022',index=False)

for doc in docs:
  arquivo = pd.DataFrame()
  for ano in range(2011,2022+1):
    arquivo = pd.concat([arquivo, pd.read_csv(f'DFP/dfp_cia_aberta_{doc}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'Dados/dfp_cia_aberta_{doc}_2011-2022',index=False)

Carregando data frames

In [ ]:
BPA_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_BPA_ind_2011-2022', sep=',', decimal=',')
BPP_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_BPP_ind_2011-2022', sep=',', decimal=',')
DVA_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_DVA_ind_2011-2022', sep=',', decimal=',')
DRE_itr = pd.read_csv(f'/content/Dados/itr_cia_aberta_DRE_ind_2011-2022', sep=',', decimal=',')

BPA_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_BPA_ind_2011-2022', sep=',', decimal=',')
BPP_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_BPP_ind_2011-2022', sep=',', decimal=',')
DVA_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_DVA_ind_2011-2022', sep=',', decimal=',')
DRE_dfp = pd.read_csv(f'/content/Dados/dfp_cia_aberta_DRE_ind_2011-2022', sep=',', decimal=',')

#Reescrevendo empresas que mudaram de nome ao longo dos anos.

In [ ]:
dataframes = [BPA_itr, BPP_itr, DVA_itr, DRE_itr, BPA_dfp, BPP_dfp, DVA_dfp, DRE_dfp]

for df in dataframes:
    df['DENOM_CIA'] = df['DENOM_CIA'].str.replace('SENIOR SOLUTION S.A.', 'SINQIA S.A.')

<ipython-input-9-caace5e2e005>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DENOM_CIA'] = df['DENOM_CIA'].str.replace('SENIOR SOLUTION S.A.', 'SINQIA S.A.')


#Filtrando dados

filtrandos os dados trimestrais relevantes para BPA

In [ ]:
BPA_itr = BPA_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)
#BPA_itr = BPA_itr[BPA_itr['CD_CONTA'].str.len() <= 7]

#reodena colunas
BPA_itr = BPA_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Ativo Total','Ativo Circulante','Disponibilidades','Caixa e Equivalentes de Caixa']
valores_a_manter = ['1','1.01','1.01.01']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPA_itr = BPA_itr.loc[BPA_itr['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPA_itr = BPA_itr[BPA_itr['ORDEM_EXERC'] == 'ÚLTIMO']
BPA_itr = BPA_itr.drop(['ORDEM_EXERC'],axis=1)

In [ ]:
BPA_itr['CNPJ_CIA'].value_counts()

02.291.077/0001-93    111
00.192.769/0001-59    111
16.590.234/0001-76    111
15.101.405/0001-93    111
73.903.718/0001-61    111
                     ... 
08.887.330/0001-52      3
09.347.916/0001-97      3
03.468.802/0001-19      3
51.468.056/0001-06      3
60.394.723/0001-44      3
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [ ]:
BPA_dfp = BPA_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)
#BPA_itr = BPA_itr[BPA_itr['CD_CONTA'].str.len() <= 7]

#reodena colunas
BPA_dfp = BPA_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Ativo Total','Ativo Circulante','Disponibilidades','Caixa e Equivalentes de Caixa']
valores_a_manter = ['1','1.01','1.01.01']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPA_dfp = BPA_dfp.loc[BPA_dfp['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
BPA_dfp = BPA_dfp[BPA_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
BPA_dfp = BPA_dfp.drop(['ORDEM_EXERC'],axis=1)

In [ ]:
BPA_dfp['CNPJ_CIA'].value_counts()

50.746.577/0001-15    39
61.082.004/0001-50    39
00.000.000/0001-91    36
17.314.329/0001-20    36
30.306.294/0001-45    36
                      ..
29.388.352/0001-48     3
33.000.118/0001-79     3
33.147.315/0001-15     3
03.392.498/0001-73     3
81.616.807/0001-55     3
Name: CNPJ_CIA, Length: 1093, dtype: int64

filtrandos os dados trimestrais relevantes para BPP

In [ ]:
BPP_itr = BPP_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
BPP_itr = BPP_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Passivo Total','Passivo Circulante'']
valores_a_manter = ['2','2.01','2.02'] #,'2.05','2.07'
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_itr_1 = BPP_itr.loc[BPP_itr['CD_CONTA'].isin(valores_a_manter)]

#valores_a_manter = ['Patrimônio Líquido']
valores_a_manter = ['Patrimônio Líquido']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_itr_2 = BPP_itr.loc[BPP_itr['DS_CONTA'].isin(valores_a_manter)]


BPP_itr = pd.concat([BPP_itr_1,BPP_itr_2]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER','CD_CONTA'])


#separando os resultados
BPP_itr = BPP_itr[BPP_itr['ORDEM_EXERC'] == 'ÚLTIMO']
BPP_itr = BPP_itr.drop(['ORDEM_EXERC'],axis=1)

In [ ]:
BPP_itr['CNPJ_CIA'].value_counts()

00.192.769/0001-59    148
02.397.080/0001-96    148
16.590.234/0001-76    148
15.101.405/0001-93    148
02.291.077/0001-93    148
                     ... 
33.066.408/0001-15      4
67.915.785/0001-01      4
09.123.654/0001-87      4
09.347.916/0001-97      4
26.643.756/0001-42      4
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [ ]:
BPP_dfp = BPP_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
BPP_dfp = BPP_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
#valores_a_manter = ['Passivo Total','Passivo Circulante'']
valores_a_manter = ['2','2.01','2.02'] #,'2.05','2.07'
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_dfp_1 = BPP_dfp.loc[BPP_dfp['CD_CONTA'].isin(valores_a_manter)]

#valores_a_manter = ['Patrimônio Líquido']
valores_a_manter = ['Patrimônio Líquido']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
BPP_dfp_2 = BPP_dfp.loc[BPP_dfp['DS_CONTA'].isin(valores_a_manter)]


BPP_dfp = pd.concat([BPP_dfp_1,BPP_dfp_2]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER','CD_CONTA'])

#separando os resultados
BPP_dfp = BPP_dfp[BPP_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
BPP_dfp = BPP_dfp.drop(['ORDEM_EXERC'],axis=1)

In [ ]:
BPP_dfp['CNPJ_CIA'].value_counts()

61.082.004/0001-50    52
50.746.577/0001-15    52
00.000.000/0001-91    48
16.404.287/0001-55    48
16.590.234/0001-76    48
                      ..
02.312.666/0001-00     4
02.312.699/0001-50     4
60.631.090/0001-40     4
32.803.503/0001-91     4
24.410.913/0001-44     4
Name: CNPJ_CIA, Length: 1093, dtype: int64

filtrandos os dados trimestrais relevantes para DVA

In [ ]:
DVA_itr = DVA_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DVA_itr = DVA_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Juros sobre o Capital Próprio','Dividendos']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DVA_itr = DVA_itr.loc[DVA_itr['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
DVA_itr = DVA_itr[DVA_itr['ORDEM_EXERC'] == 'ÚLTIMO']
DVA_itr = DVA_itr.drop(['ORDEM_EXERC'],axis=1)

In [ ]:
DVA_itr['CNPJ_CIA'].value_counts()

00.336.701/0001-04    84
01.104.937/0001-70    77
45.987.245/0001-92    76
00.359.742/0001-08    76
02.291.077/0001-93    74
                      ..
67.915.785/0001-01     2
09.123.654/0001-87     2
09.347.916/0001-97     2
11.306.912/0001-02     2
13.552.070/0001-02     2
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [ ]:
DVA_dfp = DVA_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DVA_dfp = DVA_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['Juros sobre o Capital Próprio','Dividendos']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DVA_dfp = DVA_dfp.loc[DVA_dfp['DS_CONTA'].isin(valores_a_manter)]

#separando os resultados
DVA_dfp = DVA_dfp[DVA_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
DVA_dfp = DVA_dfp.drop(['ORDEM_EXERC'],axis=1)

In [ ]:
DVA_dfp['CNPJ_CIA'].value_counts()

01.104.937/0001-70    36
00.359.742/0001-08    30
59.789.545/0001-71    28
71.476.527/0001-35    27
61.082.004/0001-50    26
                      ..
01.685.903/0001-16     2
84.429.752/0001-62     2
01.832.635/0001-18     2
10.265.949/0001-77     2
81.616.807/0001-55     2
Name: CNPJ_CIA, Length: 1091, dtype: int64


filtrandos os dados trimestrais relevantes para DRE

In [ ]:
DRE_itr = DRE_itr.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DRE_itr = DRE_itr[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_INI_EXERC','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['3.01','3.03','3.05','3.07','3.11','3.13','3.99']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DRE_itr = DRE_itr.loc[DRE_itr['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
DRE_itr = DRE_itr[DRE_itr['ORDEM_EXERC'] == 'ÚLTIMO']
DRE_itr = DRE_itr.drop(['ORDEM_EXERC'],axis=1)

In [ ]:
DRE_itr['CNPJ_CIA'].value_counts()

30.306.294/0001-45    427
15.144.017/0001-90    420
65.654.303/0001-73    420
62.144.175/0001-20    420
60.872.504/0001-23    420
                     ... 
26.643.756/0001-42      6
13.552.070/0001-02      6
03.468.802/0001-19      6
09.123.654/0001-87      6
60.394.723/0001-44      6
Name: CNPJ_CIA, Length: 1121, dtype: int64

In [ ]:
DRE_dfp = DRE_dfp.drop(['VERSAO','GRUPO_DFP','MOEDA','ST_CONTA_FIXA'],axis=1)

#reodena colunas
DRE_dfp = DRE_dfp[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','DT_INI_EXERC','DT_FIM_EXERC', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA','ORDEM_EXERC']]

# Defina a lista de valores que deseja manter
valores_a_manter = ['3.01','3.03','3.05','3.07','3.11','3.13','3.99']
# Mantenha as linhas que contêm os valores da lista na coluna 'DS_CONTA' e exclua as demais
DRE_dfp = DRE_dfp.loc[DRE_dfp['CD_CONTA'].isin(valores_a_manter)]

#separando os resultados
DRE_dfp = DRE_dfp[DRE_dfp['ORDEM_EXERC'] == 'ÚLTIMO']
DRE_dfp = DRE_dfp.drop(['ORDEM_EXERC','DT_INI_EXERC'],axis=1)

In [ ]:
DRE_dfp['CNPJ_CIA'].value_counts()

30.306.294/0001-45    84
15.144.017/0001-90    84
91.669.747/0001-92    84
59.285.411/0001-13    84
60.872.504/0001-23    84
                      ..
33.000.118/0001-79     6
03.392.498/0001-73     6
03.076.274/0001-52     6
51.468.056/0001-06     6
81.616.807/0001-55     6
Name: CNPJ_CIA, Length: 1092, dtype: int64

Padronizando a DRE ITR para resultados acumulados

In [ ]:
# Converter as colunas datas para o tipo datetime
DRE_itr['DT_INI_EXERC'] = pd.to_datetime(DRE_itr['DT_INI_EXERC'])
DRE_itr['DT_FIM_EXERC'] = pd.to_datetime(DRE_itr['DT_FIM_EXERC'])
DRE_itr['DT_REFER'] = pd.to_datetime(DRE_itr['DT_REFER'])
# Calcular a diferença de tempo entre DT_INI_EXERC e DT_FIM_EXERC
DRE_itr['PERIODO'] = DRE_itr['DT_FIM_EXERC'] - DRE_itr['DT_INI_EXERC']
#Reogarinzando a base
DRE_itr = DRE_itr.sort_values(['CNPJ_CIA','DENOM_CIA','CD_CVM','DT_REFER'])
DRE_itr = DRE_itr.reset_index(drop=True)
# Calcular a diferença de um resultado para o outro
DRE_itr['PERIODO_DIF'] = DRE_itr.groupby(['CNPJ_CIA','DT_REFER'])['PERIODO'].diff()
# Substituindo 'NaT' por '0 days'
DRE_itr['PERIODO_DIF'] = DRE_itr['PERIODO_DIF'].fillna(pd.Timedelta(0))
# Eliminar as linhas de resultados trimestrais e manter os acumulados (como nos demais documentos)
DRE_itr = DRE_itr[DRE_itr['PERIODO_DIF'] >= '-5 days' ]
DRE_itr = DRE_itr.drop(['PERIODO_DIF','PERIODO'], axis=1)
DRE_itr = DRE_itr.drop('DT_INI_EXERC', axis = 1)

In [ ]:
#DRE_itr[(DRE_itr['CNPJ_CIA'] == '32.372.085/0001-25') & (DRE_itr['DT_REFER'] >= '2019-03-31')].head(10)

In [ ]:
#DRE_itr[(DRE_itr['PERIODO_DIF'] < '-6 days') & (DRE_itr['PERIODO_DIF'] >= '-31 days')].head(60)

#Concatenando as bases de dados

In [ ]:
df1 = pd.concat([BPA_itr,BPP_itr]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df2 = pd.concat([DRE_itr,DVA_itr]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df_itr = pd.concat([df1,df2]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])

df_itr['DT_FIM_EXERC']= pd.to_datetime(df_itr['DT_FIM_EXERC'])
df_itr['DT_REFER']= pd.to_datetime(df_itr['DT_REFER'])

df_itr = df_itr.sort_values(['CNPJ_CIA','DT_REFER'])

<ipython-input-29-03f810631ad7>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df2 = pd.concat([DRE_itr,DVA_itr]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])


In [ ]:
df3 = pd.concat([BPA_dfp,BPP_dfp]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df4 = pd.concat([DRE_dfp,DVA_dfp]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])
df_dfp = pd.concat([df3,df4]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])

df_dfp['DT_FIM_EXERC']= pd.to_datetime(df_dfp['DT_FIM_EXERC'])
df_dfp['DT_REFER']= pd.to_datetime(df_dfp['DT_REFER'])

df_dfp = df_dfp.sort_values(['CNPJ_CIA','DT_REFER'])

In [ ]:
df = pd.concat([df_itr,df_dfp]).sort_values(by=['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM','DT_REFER','CD_CONTA'])
df = df.reset_index(drop=True)
df = df.drop_duplicates()

mesclando, padronizando e removendo nomes irrelevantes do balanço

In [ ]:
df['DS_CONTA'].value_counts()

Patrimônio Líquido                                                    31418
Passivo Total                                                         31415
Dividendos                                                            31400
Ativo Total                                                           31383
Juros sobre o Capital Próprio                                         31341
Passivo Circulante                                                    31231
Ativo Circulante                                                      31218
Lucro/Prejuízo do Período                                             31193
Caixa e Equivalentes de Caixa                                         29687
Passivo Não Circulante                                                29521
Resultado Antes do Resultado Financeiro e dos Tributos                29483
Resultado Bruto                                                       29481
Resultado Antes dos Tributos sobre o Lucro                            29362
Lucro por Aç

In [ ]:
#df.loc[df['DS_CONTA'] == 'Outras Receitas e Despesas Operacionais']

In [ ]:
df['DS_CONTA'] = df['DS_CONTA'].replace({

    'Passivos Financeiros ao Custo Amortizado' : 'Passivo Circulante',

    'Caixa e equivalentes de caixa': 'Caixa', #LOJAS HERING S.A.
    'Disponibilidades': 'Caixa',

    'Lucro por Ação - (Reais / Ação)': 'Lucro por Ação',
    'Lucro por Ação - (R$ / Ação)': 'Lucro por Ação',
    'Lucro por Ação (R$/Ação)' : 'Lucro por Ação',

    'Lucro ou Prejuízo Líquido do Período' : 'Lucro/Prejuízo do Período',

    'a': 'Receitas',
    'D': 'Receitas',
    'Receita Bruta de Venda de Bens e/ou Serviços' : 'Receitas',
    'Receita de Venda de Bens e/ou Serviços' : 'Receitas',
    'Receita' : 'Receitas',
    'Receitas das Operações' : 'Receitas',
    'Receitas da Intermediação Financeira' : 'Receitas',
    'Receitas de Intermediação Financeira' : 'Receitas',

    'Resultado Antes Tributação/Participações' : 'Resultado Antes dos Tributos sobre o Lucro',
    'Lucro ou Prejuízo das Operações Continuadas' : 'Resultado Antes dos Tributos sobre o Lucro',
    'Resultado Líquido das Operações Continuadas' : 'Resultado Antes dos Tributos sobre o Lucro',

    'Resultado Operacional' : 'Resultado Antes do Resultado Financeiro e dos Tributos',
    'Resultado antes dos Tributos sobre o Lucro' : 'Resultado Antes do Resultado Financeiro e dos Tributos',

    'Resultado Bruto Intermediação Financeira' : 'Resultado Bruto',
    'Resultado Bruto de Intermediação Financeira' : 'Resultado Bruto',

    '0': 'Ativo Total',
    'A': 'Ativo Total',
    'Ativo total:': 'Ativo Total',
    'Ativo': 'Ativo Total',
})

In [ ]:
df = df[~df['DS_CONTA'].isin(['Reversão dos Juros sobre Capital Próprio','Passivos Financeiros Avaliados ao Valor Justo através do Resultado','Outros Passivos',
                              'Passivo Exigível a Longo Prazo','Outras Receitas e Despesas Operacionais'])]

In [ ]:
# Substituir o conteúdo da coluna 'DS_CONTA' para 'caixa' onde 'CD_CONTA' for igual a '1.01'
df.loc[df['CD_CONTA'] == '1.01', 'DS_CONTA'] = 'Ativo Circulante'
df.loc[df['CD_CONTA'] == '1.01.01', 'DS_CONTA'] = 'Caixa'

In [ ]:
df = df.loc[df['DS_CONTA'] != 'Passivo Não Circulante']

Padronizandos os nomes dos balanços

In [ ]:
df['DS_CONTA'] = df['DS_CONTA'].replace({
    'Resultado Antes do Resultado Financeiro e dos Tributos': 'EBITDA',
    'Resultado Antes dos Tributos sobre o Lucro': 'EBIT',
    'Juros sobre o Capital Próprio': 'JCP',
    'Lucro/Prejuízo do Período': 'Lucro Líquido',
    'Resultado Bruto': 'Lucro Bruto',
    'Lucro/Prejuízo do Período': 'Lucro',
    'Receitas': 'Receita Total'

})

In [ ]:
df['DS_CONTA'].value_counts()

Patrimônio Líquido    31418
Passivo Circulante    31416
Passivo Total         31415
Ativo Total           31403
Ativo Circulante      31403
Caixa                 31403
Dividendos            31400
EBITDA                31378
EBIT                  31378
Lucro                 31378
Lucro Bruto           31376
Receita Total         31375
Lucro por Ação        31373
JCP                   31341
Name: DS_CONTA, dtype: int64

#Transpor a coluna DS_CONTA para colunas individuais

eliminar colunas que não serão mais necessárias, mas para não perder informação da coluna "ESCALA_MOEDA" é necessário multiplicar ou divididir a coluna "VL_CONTA"

In [ ]:
df['VL_CONTA'] = df['VL_CONTA'].astype('float64')
pd.set_option('display.float_format', '{:.4f}'.format)
# Mapeamento para fatores multiplicativos
escala_mapping = {'UNIDADE': 1000}

# Função personalizada para multiplicar por fator de escala
def multiply_by_scale(row):
    escala = row['ESCALA_MOEDA']
    if escala in escala_mapping:
        return row['VL_CONTA'] / escala_mapping[escala]
    else:
        return row['VL_CONTA']

# Aplicar a função personalizada na coluna 'VL_CONTA'
df['VL_CONTA'] = df.apply(multiply_by_scale, axis=1)

df = df.drop(['DT_FIM_EXERC','CD_CONTA','ESCALA_MOEDA'], axis =1)

In [ ]:
df = df.pivot_table(index=['CNPJ_CIA','DENOM_CIA','CD_CVM','DT_REFER'], columns='DS_CONTA', values='VL_CONTA' ).reset_index()
#reodena colunas
df = df[['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER', 'Patrimônio Líquido', 'Ativo Total', 'Ativo Circulante', 'Passivo Total','Passivo Circulante', 'Receita Total',
         'Lucro Bruto','EBIT', 'EBITDA', 'Lucro', 'Caixa', 'Dividendos', 'JCP', 'Lucro por Ação']]
df = df.rename_axis(None, axis='columns')

In [ ]:
#df.loc[df['DENOM_CIA'] == 'NATURA &CO HOLDING S.A.']

#Adicionando Número de ações a base de dados

Essa base foi retirada dos próprios documentos fornecido pela CVM.

In [ ]:
num_acoes = pd.read_csv('/content/numero_acoes_2011-2022.csv')
num_acoes = num_acoes.drop('Unnamed: 0', axis =1)
num_acoes = num_acoes.drop('Total', axis =1)

In [ ]:
TOTAL_ACOES = num_acoes['Acoes_ON'] + num_acoes['Acoes_PN']
ACOES_TESOURARIA = num_acoes['Tes_ON'] + num_acoes['Tes_PN']
num_acoes['FREE_FLOAT'] = ( TOTAL_ACOES - ACOES_TESOURARIA ) / TOTAL_ACOES
num_acoes['TOTAL_ACOES'] =  TOTAL_ACOES

In [ ]:
num_acoes.describe()

,CD_CVM,Acoes_ON,Acoes_PN,Tes_ON,Tes_PN,FREE_FLOAT,TOTAL_ACOES
count,30253.0000,30253.0000,30253.0000,30253.0000,30253.0000,30253.0000,30253.0000
mean,18349.9065,31245306662.2298,41695928654.4035,4500053.2950,386315.6439,0.9957,72941235316.6333
std,13062.6071,1075501519715.9868,1626586008093.4041,288512900.8954,5372610.1741,0.0319,2698272853451.6304
min,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
25%,14214.0000,91043.0000,0.0000,0.0000,0.0000,1.0000,103280.0000
50%,19771.0000,2609000.0000,0.0000,0.0000,0.0000,1.0000,2902487.0000
75%,22543.0000,152692764.0000,56630.0000,0.0000,0.0000,1.0000,176611578.0000
max,902123.0000,45035110304475.0000,68516168445415.0000,46856094000.0000,510800000.0000,1.0000,113551278749890.0000


In [ ]:
num_acoes.loc[num_acoes['FREE_FLOAT'] < 0]

,CNPJ_CIA,DT_REFER,DENOM_CIA,CD_CVM,Acoes_ON,Acoes_PN,Tes_ON,Tes_PN,FREE_FLOAT,TOTAL_ACOES
429,00.354.053/0001-00,2012-03-31,KOSMOS COMÉRCIO DE VESTUÁRIO S/A - EM RECUPERA...,15083,16083.0000,14842.0000,0.0000,509488.0000,-15.4750,30925.0000
6999,04.423.567/0001-21,2018-06-30,ENEVA S.A,21237,314990.0000,0.0000,3149900.0000,0.0000,-9.0000,314990.0000
9282,07.620.094/0001-40,2012-03-31,AFLUENTE G&T DE ENERGIA ELÉTRICA S/A,20150,10930.0000,7887.0000,17958.0000,12958.0000,-0.6430,18817.0000
9419,07.689.002/0001-89,2015-03-31,EMBRAER S.A.,20087,740465.0000,0.0000,4687483.0000,0.0000,-5.3305,740465.0000
19321,26.173.550/0001-04,2017-06-30,LGT HUB PARTICIPAÇÕES S.A.,24198,1000.0000,0.0000,49000.0000,0.0000,-48.0000,1000.0000
22807,46.119.855/0001-37,2018-03-31,SOCIEDADE ABASTECIMENTO DE ÁGUA E SANEAMENTO S...,16241,336430.0000,0.0000,3364300.0000,0.0000,-9.0000,336430.0000


substituindo resultado com linhas anteriores em que free float deu negativo

In [ ]:
num_acoes.iloc[19321] = num_acoes.iloc[19321-1]
num_acoes.iloc[429] = num_acoes.iloc[429-1]
num_acoes.iloc[6999] = num_acoes.iloc[6999-1]
num_acoes.iloc[9282] = num_acoes.iloc[9282-1]
num_acoes.iloc[9419] = num_acoes.iloc[9419-1]
num_acoes.iloc[22807] = num_acoes.iloc[22807-1]

In [ ]:
num_acoes = num_acoes[['CNPJ_CIA','DENOM_CIA','CD_CVM','DT_REFER','Acoes_ON','Acoes_PN','Tes_ON','Tes_PN','FREE_FLOAT','TOTAL_ACOES']]

Concatenando as quantidades de ações na base

In [ ]:
concated = pd.concat([df,num_acoes]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])

<ipython-input-148-594d6a0a4acf>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  concated = pd.concat([df,num_acoes]).sort_values(by=['CNPJ_CIA', 'CD_CVM','DT_REFER'])


In [ ]:
cols = ['Acoes_ON', 'Acoes_PN', 'Tes_ON', 'Tes_PN','FREE_FLOAT','TOTAL_ACOES']
for col in cols:
    concated[col] = concated.groupby('CNPJ_CIA')[col].fillna(method='Ffill')
for col in cols:
    concated[col] = concated.groupby('CNPJ_CIA')[col].fillna(method='bfill')

In [ ]:
concated = concated.dropna(subset=['Patrimônio Líquido'])
df = concated

#Filtrando a base de dados para empresas relevantes
**É possível implementar mais filtros**

In [ ]:
df['CNPJ_CIA'].value_counts()

15.101.405/0001-93    49
00.000.000/0001-91    48
16.614.075/0001-00    48
21.255.567/0001-89    48
19.527.639/0001-58    48
                      ..
40.263.170/0001-83     4
20.553.221/0001-02     4
17.162.280/0001-37     4
33.000.118/0001-79     4
11.858.621/0001-27     4
Name: CNPJ_CIA, Length: 762, dtype: int64

removendo empresas que não apresentam os resultados de no mínimo 4 trismestres (59 removidas)

In [ ]:
contagem = df['CNPJ_CIA'].value_counts()
df = df[df['CNPJ_CIA'].isin(contagem[contagem >= 4].index)]
# Iterar sobre os resultados da contagem
#for valor, count in contagem.items():
    #print(f"Valor: {valor}, Contagem: {count}")

In [ ]:
#df.loc[df['CNPJ_CIA'] == '49.263.189/0001-02']

Removendo empresas com menos de 10milhões em Ativo Total (226 removidas)

In [ ]:
df_filtrado = df[df['Ativo Total'] < 10000]
contagem = df_filtrado['CNPJ_CIA'].value_counts()
#Filtro as empresas que aparecem mais que 3x vezes para evitar balanços incosistentes
df_filtrado = df_filtrado[df_filtrado['CNPJ_CIA'].isin(contagem[contagem >= 4].index)]
#for valor, count in contagem.items():
#    print(f"Valor: {valor}, Contagem: {count}")
df = df[~df['CNPJ_CIA'].isin(df_filtrado['CNPJ_CIA'])]


Removendo empresas com menos de 100mil de Receita Total (275 removidas)

In [ ]:
#df_filtrado = df[df['Receita Total'] < 100]
#contagem = df_filtrado['CNPJ_CIA'].value_counts()
#Filtro as empresas que aparecem mais que 3x vezes para evitar balanços incosistentes
#df_filtrado = df_filtrado[df_filtrado['CNPJ_CIA'].isin(contagem[contagem >= 4].index)]
#for valor, count in contagem.items():
#    print(f"Valor: {valor}, Contagem: {count}")
#df = df[~df['CNPJ_CIA'].isin(df_filtrado['CNPJ_CIA'])]


Removendo empresas basedo no numero de ações e free float (84 empresas <= 1500 ações)

In [ ]:
df[['FREE_FLOAT','TOTAL_ACOES']].describe()

,FREE_FLOAT,TOTAL_ACOES
count,25535.0000,25535.0000
mean,0.9953,79853162534.1846
std,0.0288,2846746312406.0972
min,0.0000,1.0000
25%,0.9997,162166.0000
50%,1.0000,9137263.0000
75%,1.0000,221673074.0000
max,1.0000,113551278749890.0000


In [ ]:
pd.qcut(df['TOTAL_ACOES'], 10)

0        (1017914746.0, 113551278749890.0]
1        (1017914746.0, 113551278749890.0]
2        (1017914746.0, 113551278749890.0]
3        (1017914746.0, 113551278749890.0]
4        (1017914746.0, 113551278749890.0]
                       ...                
31378          (370000000.0, 1017914746.0]
31379          (370000000.0, 1017914746.0]
31380          (370000000.0, 1017914746.0]
31381          (370000000.0, 1017914746.0]
31382          (370000000.0, 1017914746.0]
Name: TOTAL_ACOES, Length: 25535, dtype: category
Categories (10, interval[float64, right]): [(0.999, 16027.0] < (16027.0, 90745.8] <
                                            (90745.8, 279298.0] < (279298.0, 840106.0] < ... <
                                            (66002915.0, 154207162.0] < (154207162.0, 370000000.0] <
                                            (370000000.0, 1017914746.0] < (1017914746.0, 113551278749890.0]]

10% da base contem empresas com numero de ações com menos de 16027

In [ ]:
df_filtrado = df[df['TOTAL_ACOES'] <= 15000]
contagem = df_filtrado['CNPJ_CIA'].value_counts()
#Filtro as empresas que aparecem mais que 1x vezes para evitar balanços incosistentes
df_filtrado = df_filtrado[df_filtrado['CNPJ_CIA'].isin(contagem[contagem >= 10].index)]
#for valor, count in contagem.items():
#    print(f"Valor: {valor}, Contagem: {count}")
df = df[~df['CNPJ_CIA'].isin(df_filtrado['CNPJ_CIA'])]

preenchendo valores com 0 para Dy, JCP e LPA.

In [ ]:
df[['Dividendos','JCP','Lucro por Ação']] = df[['Dividendos','JCP','Lucro por Ação']].fillna(0)

#Inserindo Indicadores em um novo DF

In [ ]:
# Vetor de indicadores
indicadores = ['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER','VPA','LPA_BASE','LPA_CALC','PAYOUT','PL/ATIVOS',
                'PS/ATIVOS','DIV/PL','DIV/EBIT','DIV/EBITDA','M.EBITDA','ROE','ROIC','ROA','ILC','MLB','MLO','MLL','VLE']
# Criar novo DataFrame com coluna de indicadores repetidos
df2 = pd.DataFrame(columns=[indicadores])
df2 = df2.assign(**{coluna: df[coluna] for coluna in ['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'DT_REFER']})

Calculando os indicadores a partir dos balanços das empresas

In [ ]:
df2['LPA_BASE'] = df['Lucro por Ação']
df2['LPA_CALC'] = (df['Lucro'] / df['TOTAL_ACOES'])*1000
df2['VPA'] = df['Patrimônio Líquido'] / ( df['TOTAL_ACOES'])*1000
df2['PAYOUT'] = ( df['Dividendos'] + df['JCP'] ) / ( df['Lucro'])

df2['PL/ATIVOS'] = df['Patrimônio Líquido'] / df['Ativo Total']
df2['PS/ATIVOS'] = df['Passivo Total'] / df['Ativo Total']
df2['DIV/PL'] = (df['Passivo Circulante'] - df['Caixa']) / df['Patrimônio Líquido']
df2['DIV/EBIT'] = (df['Passivo Circulante'] - df['Caixa']) / df['EBIT']
df2['DIV/EBITDA'] = (df['Passivo Circulante'] - df['Caixa']) / df['EBITDA']

df2['M.EBITDA'] = df['EBITDA'] / df['Receita Total']
df2['ROE'] = df['Lucro'] / df['Patrimônio Líquido']
df2['ROA'] = df['Lucro'] / df['Ativo Total']
df2['ROIC'] = df['EBIT'] / (df['Patrimônio Líquido'] + (df['Passivo Total'] - df['Passivo Circulante']))

df2['ILC'] = df['Ativo Circulante'] / df['Passivo Circulante']
df2['MLB'] = df['Lucro Bruto'] / df['Receita Total']
df2['MLO'] = df['EBIT'] / df['Receita Total']
df2['MLL'] = df['Lucro'] / df['Receita Total']

# Calcular a variação percentual do lucro
df2['VLE'] = df.groupby('CNPJ_CIA')['Lucro'].pct_change()
#df2['VLE'] = (df['Lucro'] / df['Lucro'].shift(1)) - 1

In [ ]:
df2.head(60)

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,VPA,LPA_BASE,LPA_CALC,PAYOUT,PL/ATIVOS,PS/ATIVOS,...,DIV/EBITDA,M.EBITDA,ROE,ROIC,ROA,ILC,MLB,MLO,MLL,VLE
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-03-31 00:00:00,18.2376,0.0000,1.0250,0.4000,0.0662,1.0000,...,118.6082,0.2191,0.0562,0.0146,0.0037,0.8585,0.3157,0.2197,0.1453,NaN
1,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-06-30 00:00:00,18.9728,2.1891,2.1891,0.4000,0.0665,1.0000,...,56.8123,0.2240,0.1154,0.0302,0.0077,0.8653,0.3207,0.2266,0.1488,1.1356
2,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-09-30 00:00:00,19.6730,3.2000,3.1998,0.4000,0.0655,1.0000,...,43.9160,0.1860,0.1626,0.0368,0.0107,0.8965,0.2678,0.1880,0.1356,0.4617
3,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31 00:00:00,20.3265,4.2802,4.2812,0.4000,0.0653,1.0000,...,32.6106,0.1917,0.2106,0.0464,0.0138,0.9061,0.2732,0.1937,0.1360,0.3380
4,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-03-31 00:00:00,20.8758,0.8915,0.8915,0.4000,0.0655,1.0000,...,164.7938,0.1528,0.0427,0.0088,0.0028,0.9389,0.2823,0.1539,0.1135,-0.7914
5,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-06-30 00:00:00,21.6369,1.9390,1.9390,0.4000,0.0644,1.0000,...,77.7476,0.1595,0.0896,0.0180,0.0058,0.9202,0.2576,0.1609,0.1175,1.1750
6,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-09-30 00:00:00,22.2809,2.9188,2.9177,0.4000,0.0632,1.0000,...,58.3499,0.1561,0.1310,0.0256,0.0083,0.9017,0.2658,0.1580,0.1205,0.5048
7,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-12-31 00:00:00,23.0136,4.3023,4.3069,0.4000,0.0627,1.0000,...,42.2973,0.1718,0.1871,0.0378,0.0117,0.8738,0.2771,0.1854,0.1357,0.4724
8,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-03-31 00:00:00,21.5846,0.9057,0.9014,0.4000,0.0559,1.0000,...,199.4741,0.1650,0.0418,0.0076,0.0023,0.8781,0.3213,0.1654,0.1224,-0.7902
9,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-06-30 00:00:00,21.8215,3.5212,3.5015,0.4000,0.0549,1.0000,...,145.6190,0.1065,0.1605,0.0321,0.0088,0.8772,0.2486,0.3175,0.2145,2.8843


Essa situação está ocorrendo por que essas empresas não reportaram um ou mais balanços no período. (as empresas são obrigadas a reportarem anualmente apenas)

In [ ]:
df[df.isna().any(axis=1)]

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,Patrimônio Líquido,Ativo Total,Ativo Circulante,Passivo Total,Passivo Circulante,Receita Total,...,Caixa,Dividendos,JCP,Lucro por Ação,Acoes_ON,Acoes_PN,Tes_ON,Tes_PN,FREE_FLOAT,TOTAL_ACOES
21069,31.001.230/0001-07,CELEO REDES TRANSMISSÃO DE ENERGIA S.A.,24597,2018-06-30 00:00:00,0.0000,NaN,NaN,0.0000,0.0000,NaN,...,NaN,0.0000,0.0000,0.0000,10.0000,0.0000,0.0000,0.0000,1.0000,10.0000
21284,32.785.497/0001-97,NATURA &CO HOLDING S.A.,24783,2019-06-30 00:00:00,0.1000,0.1000,0.1000,0.1000,0.0000,NaN,...,0.1000,0.0000,0.0000,0.0000,100.0000,0.0000,0.0000,0.0000,1.0000,100.0000


In [ ]:
df.head(60)

,CNPJ_CIA,DENOM_CIA,CD_CVM,DT_REFER,Patrimônio Líquido,Ativo Total,Ativo Circulante,Passivo Total,Passivo Circulante,Receita Total,...,Caixa,Dividendos,JCP,Lucro por Ação,Acoes_ON,Acoes_PN,Tes_ON,Tes_PN,FREE_FLOAT,TOTAL_ACOES
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-03-31 00:00:00,52172709.0000,787908489.0000,460670797.0000,787908489.0000,536607202.0000,20179781.0000,...,12244142.0000,449024.0000,723921.0000,0.0000,2860729247.0000,0.0000,32.0000,0.0000,1.0000,2860729247.0000
1,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-06-30 00:00:00,54276101.0000,816131152.0000,479955357.0000,816131152.0000,554643276.0000,42097733.0000,...,18868357.0000,1044346.0000,1460601.0000,2.1891,2860729247.0000,0.0000,32.0000,0.0000,1.0000,2860729247.0000
2,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-09-30 00:00:00,56279126.0000,859280120.0000,512077755.0000,859280120.0000,571192875.0000,67494104.0000,...,19767405.0000,1405060.0000,2256401.0000,3.2000,2860729247.0000,0.0000,32.0000,0.0000,1.0000,2860729247.0000
3,00.000.000/0001-91,BCO BRASIL S.A.,1023,2011-12-31 00:00:00,58148690.0000,890352257.0000,518716710.0000,890352257.0000,572481618.0000,90080002.0000,...,9227217.0000,1847625.0000,3051308.0000,4.2802,2860729247.0000,0.0000,32.0000,0.0000,1.0000,2860729247.0000
4,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-03-31 00:00:00,59817963.0000,913866693.0000,545372613.0000,913866693.0000,580836511.0000,22500822.0000,...,14265629.0000,181408.0000,840365.0000,0.8915,2865416988.0000,0.0000,32.0000,0.0000,1.0000,2865416988.0000
5,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-06-30 00:00:00,61998776.0000,962760903.0000,554627497.0000,962760903.0000,602726998.0000,47296102.0000,...,16287118.0000,531682.0000,1690693.0000,1.9390,2865416973.0000,0.0000,47.0000,0.0000,1.0000,2865416973.0000
6,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-09-30 00:00:00,63844116.0000,1009923655.0000,582005138.0000,1009923655.0000,645454593.0000,69355842.0000,...,13603652.0000,835926.0000,2508259.0000,2.9188,2865416973.0000,0.0000,47.0000,0.0000,1.0000,2865416973.0000
7,00.000.000/0001-91,BCO BRASIL S.A.,1023,2012-12-31 00:00:00,65776796.0000,1049283624.0000,585819632.0000,1049283624.0000,670456746.0000,90745179.0000,...,11189103.0000,1570156.0000,3353792.0000,4.3023,2858171973.0000,0.0000,7245047.0000,0.0000,0.9975,2858171973.0000
8,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-03-31 00:00:00,61848822.0000,1105744898.0000,620472390.0000,1105744898.0000,706574074.0000,21106633.0000,...,11974401.0000,279429.0000,753777.0000,0.9057,2865417020.0000,0.0000,20200047.0000,0.0000,0.9930,2865417020.0000
9,00.000.000/0001-91,BCO BRASIL S.A.,1023,2013-06-30 00:00:00,62527717.0000,1138813786.0000,647797545.0000,1138813786.0000,738514308.0000,46784697.0000,...,13195754.0000,2457310.0000,1556018.0000,3.5212,2865417020.0000,0.0000,20200047.0000,0.0000,0.9930,2865417020.0000


In [ ]:
#df_teste.to_csv('df_teste.csv', index=True, header = True)

In [ ]:
df.to_csv('balanço_empresas_2011-2022.csv', index=True, header = True)
df2.to_csv('indicadores_empresas_2011-2022.csv', index=True, header = True)

#Considerações

O tratamento para a mineração adequada da base de dados foi realizado de forma meticulosa, abordando todas as possibilidades. Durante o desenvolvimento, foram identificadas diversas inconsistências na mineração de dados, mas mesmo assim, foi possível alcançar o principal objetivo: garantir que não houvesse nenhum dado "NaN" de empresas relevantes na base (apenas duas empresa apresentaram "NaN"), resultante de problemas na divulgação de resultados. Vale ressaltar que, apesar disso, ainda é necessário realizar o Feature Engineering para aprimorar a base de dados.

Além disso, quanto maior e mais sólida a empresa, melhor é a qualidade dos dados. Para garantir a integridade da base, foi implementado um filtro inicial para excluir empresas muito pequenas. É importante destacar que nem todas as empresas com capital aberto estão listadas em bolsa. Após a aplicação do filtro, restaram 762 empresas, das 1131 inicialmente consideradas, no período entre 2011 e 2022. Mais filtros podem ser aplicados.

Por fim, os indicadores foram calculados para uma ideia inicial, porém, seria interessante deixá-los para depois, sendo assim, a base de dados está pronta para etapa seguinte : **Tratamentos dos dados ou Featuring Engineer.**